##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Деревья решений с использованием оценщиков(Estimators)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/boosted_trees"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/boosted_trees.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/boosted_trees.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/estimator/boosted_trees.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Это раздел представляет собой сквозное пошаговое руководство по обучению модели градиентного бустинга с использованием деревьев решений с помощью `tf.estimator`. Модели Boosted Trees являются одним из самых популярных и эффективных подходов к машинному обучению как для регрессии, так и для классификации. Это техника ансамбля, которая объединяет прогнозы из нескольких(например, 10, 100 или даже 1000) моделей решающих деревьев.

Модели Boosted Trees популярны среди многих практиков машинного обучения, поскольку они могут достичь впечатляющей производительности с минимальной настройкой гиперпараметров.

## Загрузка датасета пассажиров Титаника

Вы будете использовать набор данных "Титаник", цель которого(довольно болезненная) - предсказать выживаемость пассажиров с учетом таких характеристик, как пол, возраст, класс и т.д.

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

# Загружаем датасет.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [ ]:
import tensorflow as tf
tf.random.set_seed(123)

Набор данных состоит из обучающего и тестового набора:

* `dftrain` и` y_train` - это *обучающий датасет* - данные, которые модель использует для обучения.
* Модель протестирована с использованием *тестового датасета* - `dfeval` и` y_eval`.

Для обучения вы будете использовать следующие возможности:


<table>
  <tr>
    <th> Название функции </th>
    <th> Описание </th>
  </tr>
  <tr>
    <td> sex </td>
    <td> Пол пассажира </td>
  </tr>
  <tr>
    <td> age </td>
    <td> Возраст пассажира </td>
  </tr>
    <tr>
    <td> n_siblings_spouses </td>
    <td> братья, сестры и супргуи на борту </td>
  </tr>
    <tr>
    <td> parch </td>
    <td> родители и дети на борту </td>
  </tr>
    <tr>
    <td> fare </td>
    <td> Тариф пассажира </td>
  </tr>
    <tr>
    <td> class </td>
    <td> Класс пассажира на корабле </td>
  </tr>
    <tr>
    <td> deck </td>
    <td> На какой палубе находился пассажир </td>
  </tr>
    <tr>
    <td> embark_town </td>
    <td> В каком городе пассажир совершил посадку </td>
  </tr>
    <tr>
    <td> alone </td>
    <td> Был ли пассажир один </td>
  </tr>
</table>

## Изучение данных

Давайте сначала просмотрим некоторые данные и создадим сводную статистику по обучающей выборке.

In [ ]:
dftrain.head()

In [ ]:
dftrain.describe()

В обучающей и тестовой выборках 627 и 264 примеров соответственно.

In [ ]:
dftrain.shape[0], dfeval.shape[0]

Большинство пассажиров в возрасте от 20 до 30 лет.

In [ ]:
dftrain.age.hist(bins=20)
plt.show()

Пассажиров-мужчин примерно в два раза больше, чем пассажиров-женщин.

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')
plt.show()

Большинство пассажиров находились в «третьем» классе.

In [ ]:
dftrain['class'].value_counts().plot(kind='barh')
plt.show()

Большинство пассажиров сели в Саутгемптоне.

In [ ]:
dftrain['embark_town'].value_counts().plot(kind='barh')
plt.show()

У женщин намного больше шансов выжить, чем у мужчин. Это явно будет прогностическим признаком модели.

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')
plt.show()

## Создание столбцов признаков и функций ввода

Эстиматор Gradient Boosting может использовать как числовые, так и категориальные признаки. Столбцы признаков работают со всеми эстиматорами TensorFlow, и их цель - определить признаки, используемые для моделирования. Кроме того, они предоставляют некоторые возможности разработки функций, такие как `one-hot-encoding`, `нормализация` и `разделение на пакеты`. В этом руководстве поля в `CATEGORICAL_COLUMNS` преобразуются из категориальных столбцов в `one-hot-encoding` столбцы ([столбец индикатора](https://www.tensorflow.org/api_docs/python/tf/feature_column/indicator_column)):

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

def one_hot_cat_column(feature_name, vocab):
  return tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Нужно перобразовать категориальные признаки в числовые с помощью `one-hot-encoding` метода.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))

Вы можете просмотреть преобразование, выполненное над признаками. Например, вот результат использования `indicator_column` в одном примере:

In [ ]:
example = dict(dftrain.head(1))
class_fc = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list('class', ('First', 'Second', 'Third')))
print('Feature value: "{}"'.format(example['class'].iloc[0]))
print('One-hot encoded: ', tf.keras.layers.DenseFeatures([class_fc])(example).numpy())

Кроме того, вы можете просматривать все преобразования признаков вместе:

In [ ]:
tf.keras.layers.DenseFeatures(feature_columns)(example).numpy()

Далее вам нужно создать функции ввода. Они укажут, как данные будут считываться в нашу модель как для обучения, так и для вывода. Вы будете использовать метод `from_tensor_slices` из [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) для чтения данных непосредственно из Pandas. Это подходит для небольших наборов данных, умещающихся в памяти. Для больших наборов данных `tf.data`  поддерживает различные форматы файлов(включая [csv](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)), чтобы вы могли обрабатывать наборы данных которые не помещаются в памяти.

In [ ]:
# Используйте весь пакет, так как это небольшой набор данных.
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # Для обучения повторите набор данных столько раз, сколько потребуется(n_epochs = None).
    dataset = dataset.repeat(n_epochs)
    # При обучении на датасете, помещающимся в памяти нет необходимости в разделении на пакеты.
    dataset = dataset.batch(NUM_EXAMPLES)
    return dataset
  return input_fn

# Функции ввода для обучения и оценки.
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

## Обучение и оценка модели

Далее проделайте следующие шаги:

1. Инициализируйте модель, указав функции и гиперпараметры.
2. Загрузите обучающие данные в модель с помощью `train_input_fn` и обучите модель с помощью функции `train`.
3. Вы будете оценивать производительность модели, используя оценочный набор - в этом примере DataFrame `dfeval`. Вы должны убедится, что прогнозы соответствуют меткам из массива `y_eval`.

Перед обучением модели Boosted Trees давайте сначала обучим линейный классификатор (модель логистической регрессии). Лучше всего начать с более простой модели, чтобы иметь отправную точку.

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns)

# Обучение.
linear_est.train(train_input_fn, max_steps=100)

# Оценка.
result = linear_est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

Теперь давайте обучим модель Boosted Trees. Для решающих деревьев поддерживаются регрессия(`BoostedTreesRegressor`) и классификация(`BoostedTreesClassifier`). Поскольку цель - предсказать класс - выживет пассажир или нет, вы будете использовать `BoostedTreesClassifier`.

In [ ]:
# Поскольку данные помещаются в память, используйте весь набор данных для каждого слоя. Так будет быстрее.
# Выше мы определили размер пакета равным размеру всего датасета.
n_batches = 1
est = tf.estimator.BoostedTreesClassifier(feature_columns,
                                          n_batches_per_layer=n_batches)

# Модель прекратит обучение после того, как будет построено указанное 
# количество деревьев, а не количество шагов.
est.train(train_input_fn, max_steps=100)

# Оценка.
result = est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

Теперь вы можете использовать обученную модель, чтобы делать прогнозы относительно пассажиров из тестового датасета. Модели TensorFlow могут делать прогнозы для пакета, коллекции или единичного примера. Ранее мы определили использование всего тестового датасета для`eval_input_fn`.

In [ ]:
pred_dicts = list(est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')
plt.show()

И наконец, вы можете посмотреть на ROC-кривую, позволяющий оценить качество нашей классификации.

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)
plt.show()